# Convert Facebook sticker to GIF

Bạn cần có:
- python 3
- `pip install pillow`
- Sticker đã download từ facebook (để trong thư mục **input**), có dạng tên file như sau:  
`(StickerID)_(Tổng số cột)_(Tổng số hàng)_(Tổng số ảnh)_(Frame rate).png`

In [22]:
from PIL import Image
from PIL import GifImagePlugin
from PIL import ImageSequence
from os import listdir
from os.path import isfile, join
from IPython.display import display

In [26]:
img = Image.open('1.webp').convert('RGBA')

for i, frame in enumerate(ImageSequence.Iterator(img)):
    img.save(str(i)+'.gif', 'GIF', save_all=True)

In [2]:
def generateCoords(tcol, trow, w, h):
    out = []
    dx = w // tcol
    dy = h // trow
    for row in range(0, trow):
        for col in range(0, tcol):
            x1 = col * dx
            y1 = row * dy
            x2 = x1 + dx
            y2 = y1 + dy
            out.append((x1 + 2, y1 + 2, x2 - 2, y2 - 2))
    return (out, dx, dy)

In [3]:
def expandImage(image):
    w, h = image.size
    background = Image.new('RGB', (w+10, h+10), 'WHITE')
    background.paste(image, (0,0), image)
    return background

In [4]:
def splitImage(image, col, row, frames):
    w, h = image.size
    paddedImage = expandImage(image)
    (crop_spec, dw, dh) = generateCoords(col, row, w, h)
    imgs = [ paddedImage.crop(tup) for tup in crop_spec ][:frames]
    nimgs = []
    bg_w = dw * 110 // 100
    bg_h = dh * 110 // 100
    offset = ((bg_w - dw) // 2, (bg_h - dh) // 2)
    bgSize = (bg_w, bg_h)
    for i, img in enumerate(imgs):
        background = Image.new('RGB', bgSize, 'WHITE')
        background.paste(img, offset)
        nimgs.append(background)
    return nimgs

In [5]:
def makeGif(img, stickerId, col, row, frames, rate):
    imageArr = splitImage(img, col, row, frames)
    firstImage = imageArr[0]
    del imageArr[0]
    newFileName = './output/' + stickerId + '.gif'
    firstImage.save(newFileName, 'GIF', save_all=True, append_images=imageArr, duration=rate, loop=0)

In [6]:
for filename in listdir('./input'):
    filePath = join('./input', filename)
    if not isfile(filePath):
        continue
    img = Image.open(filePath).convert('RGBA')
    param = filename.replace('.png', '').split('_')
    makeGif(img, param[0], int(param[1]), int(param[2]), int(param[3]), int(param[4]))

In [7]:
#img = Image.open('./2041015576125916_5_4_20_83.png').convert('RGBA')
#makeGif(img, '2041015576125916', 5, 4, 20, 83)